In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

%matplotlib inline
tf.config.set_visible_devices([], 'GPU')

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (150, 150))
            img_data = np.asarray(img)
            images.append(img_data)
    return images

In [3]:
def load_images_from_folder_2(folder):
    images = []
    for filename in os.listdir(folder):
        loaded_img = load_img(os.path.join(folder,filename), target_size=(150, 150))
        numpy_img = img_to_array(loaded_img)
        images.append(numpy_img)
    return images

In [4]:
control_images = load_images_from_folder_2('CT_NonCOVID')
covid_images = load_images_from_folder_2('CT_COVID')
all_images = control_images + covid_images
all_images = np.array(all_images)
# all_images = all_images.reshape((-1, 150, 150, 3))

In [5]:

all_images.shape

(746, 150, 150, 3)

In [6]:
# for i in range(4):
#     plt.subplot(2, 2, i + 1)
#     plt.imshow(np.uint8(all_images[i]))
# plt.show

In [7]:
control_labels = [0] * len(control_images)
covid_labels = [1] * len(covid_images)
label_series = pd.Series(control_labels + covid_labels)
print(len(all_images))
print(len(label_series))

746
746


In [8]:
X_train, X_test, y_train, y_test = train_test_split(all_images, label_series, test_size=0.25, random_state=42)

In [9]:
print(len(X_train))
print(len(y_train))

559
559


In [10]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=(150, 150, 3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


In [11]:
model.fit(X_train, y_train, batch_size=64, epochs=8, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()
print('COVID image detection accuracy:', ba.result().numpy())
print('COVID image detection recall:', recall_result)
print('COVID image detection precision:', precision_result)
print('COVID image detection F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 18s 32ms/sample - loss: 5.0330 - accuracy: 0.4830
Epoch 2/8
559/559 [==============================] - 17s 31ms/sample - loss: 1.1087 - accuracy: 0.5116
Epoch 3/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.8248 - accuracy: 0.4884
Epoch 4/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.7103 - accuracy: 0.5599
Epoch 5/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.6790 - accuracy: 0.5671
Epoch 6/8
559/559 [==============================] - 18s 31ms/sample - loss: 0.6437 - accuracy: 0.6154
Epoch 7/8
559/559 [==============================] - 17s 31ms/sample - loss: 0.6398 - accuracy: 0.6172
Epoch 8/8
559/559 [==============================] - 18s 31ms/sample - loss: 0.6013 - accuracy: 0.6494
COVID image detection accuracy: 0.7379679
COVID image detection recall: 0.6363636
COVID image detection precision: 0.7
COVID image detection F1 score: 0.66

In [12]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=(150, 150, 3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, batch_size=64, epochs=10, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()
print('COVID image detection accuracy:', ba.result().numpy())
print('COVID image detection recall:', recall_result)
print('COVID image detection precision:', precision_result)
print('COVID image detection F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/10
559/559 [==============================] - 5s 9ms/sample - loss: 50.6849 - accuracy: 0.4687
Epoch 2/10
559/559 [==============================] - 5s 8ms/sample - loss: 14.9386 - accuracy: 0.5331
Epoch 3/10
559/559 [==============================] - 5s 8ms/sample - loss: 7.4654 - accuracy: 0.5170
Epoch 4/10
559/559 [==============================] - 5s 8ms/sample - loss: 4.1265 - accuracy: 0.5546
Epoch 5/10
559/559 [==============================] - 5s 8ms/sample - loss: 2.3619 - accuracy: 0.5832
Epoch 6/10
559/559 [==============================] - 5s 8ms/sample - loss: 1.0948 - accuracy: 0.6404
Epoch 7/10
559/559 [==============================] - 5s 8ms/sample - loss: 0.7818 - accuracy: 0.7013
Epoch 8/10
559/559 [==============================] - 5s 8ms/sample - loss: 0.5658 - accuracy: 0.7299
Epoch 9/10
559/559 [==============================] - 5s 8ms/sample - loss: 0.4171 - accuracy: 0.8229
Epoch 10/10
559/559 [==============================] - 5s 8

In [20]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [22]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train, batch_size=64, epochs=20, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()
print('COVID image detection accuracy:', ba.result().numpy())
print('COVID image detection recall:', recall_result)
print('COVID image detection precision:', precision_result)
print('COVID image detection F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/20
559/559 [==============================] - 15s 26ms/sample - loss: 0.7813 - accuracy: 0.4937
Epoch 2/20
559/559 [==============================] - 13s 23ms/sample - loss: 0.7536 - accuracy: 0.4991
Epoch 3/20
559/559 [==============================] - 13s 23ms/sample - loss: 0.7352 - accuracy: 0.5098
Epoch 4/20
559/559 [==============================] - 13s 23ms/sample - loss: 0.7197 - accuracy: 0.5349
Epoch 5/20
559/559 [==============================] - 13s 23ms/sample - loss: 0.7027 - accuracy: 0.5599
Epoch 6/20
559/559 [==============================] - 13s 23ms/sample - loss: 0.6913 - accuracy: 0.5689
Epoch 7/20
559/559 [==============================] - 13s 23ms/sample - loss: 0.6873 - accuracy: 0.5778
Epoch 8/20
559/559 [==============================] - 13s 23ms/sample - loss: 0.6828 - accuracy: 0.5957
Epoch 9/20
559/559 [==============================] - 13s 23ms/sample - loss: 0.6653 - accuracy: 0.6011
Epoch 10/20
559/559 [======================

In [24]:
model.predict(X_test)


array([[0.47928703],
       [0.45454565],
       [0.8345722 ],
       [0.5187044 ],
       [0.4411578 ],
       [0.6497623 ],
       [0.5447581 ],
       [0.52973294],
       [0.52400273],
       [0.6761935 ],
       [0.41139874],
       [0.6566758 ],
       [0.52835655],
       [0.39182684],
       [0.64127403],
       [0.40779194],
       [0.5805511 ],
       [0.55425096],
       [0.49479797],
       [0.32347217],
       [0.32929355],
       [0.64331466],
       [0.38785052],
       [0.2428097 ],
       [0.66755563],
       [0.6276143 ],
       [0.4722139 ],
       [0.7590976 ],
       [0.7889742 ],
       [0.41401467],
       [0.6170989 ],
       [0.6563938 ],
       [0.43494776],
       [0.77768534],
       [0.54579896],
       [0.46018332],
       [0.34180865],
       [0.41675484],
       [0.49826494],
       [0.5901676 ],
       [0.66189754],
       [0.6580004 ],
       [0.70505655],
       [0.28125402],
       [0.68673235],
       [0.57433206],
       [0.70576346],
       [0.479

In [25]:
base_model = tf.keras.applications.ResNet50V2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 77, 77, 64)   0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [26]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
model.fit(X_train, y_train, batch_size=64, epochs=20, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
ba.update_state(y_test, model.predict(X_test))
recall.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
precision.update_state(y_test, [prediction[0] for prediction in model.predict(X_test)])
recall_result = recall.result().numpy()
precision_result = precision.result().numpy()
print('COVID image detection accuracy:', ba.result().numpy())
print('COVID image detection recall:', recall_result)
print('COVID image detection precision:', precision_result)
print('COVID image detection F1 score:', 2 * (precision_result * recall_result) / (precision_result + recall_result))

Train on 559 samples
Epoch 1/20
559/559 [==============================] - 21s 37ms/sample - loss: 0.7040 - accuracy: 0.5420
Epoch 2/20
559/559 [==============================] - 20s 35ms/sample - loss: 0.6919 - accuracy: 0.5868
Epoch 3/20
559/559 [==============================] - 19s 35ms/sample - loss: 0.6682 - accuracy: 0.6136
Epoch 4/20
559/559 [==============================] - 19s 35ms/sample - loss: 0.6681 - accuracy: 0.6011
Epoch 5/20
559/559 [==============================] - 20s 35ms/sample - loss: 0.6643 - accuracy: 0.6047
Epoch 6/20
559/559 [==============================] - 19s 35ms/sample - loss: 0.6445 - accuracy: 0.6404
Epoch 7/20
559/559 [==============================] - 19s 35ms/sample - loss: 0.6288 - accuracy: 0.6637
Epoch 8/20
559/559 [==============================] - 20s 35ms/sample - loss: 0.6249 - accuracy: 0.6547
Epoch 9/20
559/559 [==============================] - 20s 35ms/sample - loss: 0.6196 - accuracy: 0.6673
Epoch 10/20
559/559 [======================

C:\Users\bruce\miniconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
